# Pipelines, grid search i text mining
**Descripció**
Comencem a familiaritzar-nos amb Pipelines, grid search i text mining !!! Comencem amb uns quants exercicis bàsics

### Recursos
Recursos de l'aula i https://www.nltk.org

## Nivell 1
### Exercici 1
Agafa el conjunt de dades que vulguis i realitza un pipeline i un gridsearch aplicant l'algorisme de Random Forest.

### Exercici 2
Agafa un text en anglès que vulguis, i calcula'n la freqüència de les paraules

## Nivell 2
### Exercici 1
Treu les stopwords i realitza stemming al teu conjunt de dades.

## Nivell 3
### Exercici 1
Realitza sentiment analysis al teu conjunt de dades.